In [22]:
from typing import Annotated, Literal, TypedDict, List, Dict, Any, Optional 
from langgraph.graph import StateGraph,MessagesState, START, END
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage,AIMessage, SystemMessage
from langchain_core.tools import tool 
from langchain_groq import ChatGroq
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
import json
import os
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from docx import Document
import tempfile
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
class AgentState(TypedDict):
    messages: Annotated[list[Any], "The message in the conversation"]
    original_query: str
    research_results: Dict[str, Any]
    summary_results: Dict[str, Any]
    final_output: str
    routing_decision: Optional[str]
    task_completed: bool

In [24]:
llm  = ChatGroq(model="gemma2-9b-it", temperature=0)


In [25]:
def initialize_tools():
    """
    Initialize the tools for the agent.
    """
    #Medical Research Tool - PudMed
    pubmed_tool = PubmedQueryRun()

    #Financial Research Tools
    tavily_tool = TavilySearchResults(
        max_results =5,
        search_depth = "advanced",
        include_answer =True, 
        include_raw_content =True
    )
    wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=2000)
    wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

    return {
        "pubmed": pubmed_tool,
        "tavily": tavily_tool,
        "wikipedia": wikipedia_tool
    }

In [26]:
#initialize tools
tools = initialize_tools()

In [27]:
#Document creation tools
@tool
def create_pdf_tool(title: str, content: str) -> str:
    """
    Create a PDF document with the given title and content.
    """
    try:
        with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as tmp_file:
            doc = SimpleDocTemplate(tmp_file.name, pagesize=letter)
            styles = getSampleStyleSheet()
            story =[]
            # Add title
            story.append(Paragraph(title, styles['Title']))
            story.append(Spacer(1, 12))


            # Add content
            paragraphs = content.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    story.append(Paragraph(para, styles['Normal']))
                    story.append(Spacer(1, 12))
            
            doc.build(story)
            
            return f"PDF successfully created: {tmp_file.name}\nTitle: {title}\nContent length: {len(content)} characters"
    except Exception as e:
        return f"PDF creation failed: {str(e)}. Fallback: PDF document would contain:\nTitle: {title}\nContent: {content[:200]}..."


In [28]:
@tool
def create_word_tool(title: str, content: str) -> str:
    """Create an actual Word document from content"""
    try:
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix='.docx', delete=False) as tmp_file:
            doc = Document()
            
            # Add title
            doc.add_heading(title, 0)
            
            # Add content
            paragraphs = content.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    doc.add_paragraph(para)
            
            doc.save(tmp_file.name)
            
            return f"Word document successfully created: {tmp_file.name}\nTitle: {title}\nContent length: {len(content)} characters"
    except Exception as e:
        return f"Word creation failed: {str(e)}. Fallback: Word document would contain:\nTitle: {title}\nContent: {content[:200]}..."


In [29]:
@tool
def create_summary_tool(research_data: str) -> str:
    """Create executive summary from research results"""
    return f"""
EXECUTIVE SUMMARY
=================

Based on comprehensive research analysis, the following key findings have been identified:

{research_data}

RECOMMENDATIONS:
- Implement evidence-based strategies derived from research findings
- Consider multi-faceted approach incorporating all identified factors
- Monitor developments in relevant research areas for updates
- Ensure compliance with applicable regulations and best practices

This executive summary provides stakeholders with essential insights for informed decision-making.
"""

In [30]:
# Routing tools for intelligent decision making
@tool
def route_to_research_team(query: str) -> str:
    """Route to research team (Team1) for research-related queries"""
    return f"ROUTING_DECISION: team1 - Query '{query}' requires research capabilities"

@tool
def route_to_reporting_team(query: str) -> str:
    """Route to reporting team (Team2) for document creation and reporting"""
    return f"ROUTING_DECISION: team2 - Query '{query}' requires reporting capabilities"

@tool
def route_to_medical_research(query: str) -> str:
    """Route to medical research specialist (Team3) for medical/pharmaceutical queries"""
    return f"ROUTING_DECISION: team3 - Query '{query}' requires medical research expertise"

@tool
def route_to_financial_research(query: str) -> str:
    """Route to financial research specialist (Team4) for financial/economic queries"""
    return f"ROUTING_DECISION: team4 - Query '{query}' requires financial research expertise"


In [31]:
# Pure Agent Nodes (only handle their specific tasks)
class SupervisorAgent:
    def __init__(self):
        self.name = "Supervisor"
        self.system_message = """You are an intelligent routing supervisor. 
        
        Analyze the user's query and determine the best routing path:
        
        1. If the query requires research (medical, financial, market analysis, clinical studies, etc.), 
           use route_to_research_team.
        2. If the query is asking for document creation, reports, or summaries from existing data,
           use route_to_reporting_team.
        
        Consider the primary intent of the query, not just keywords. For example:
        - "Analyze market impact of new Alzheimer's drug" → research team (needs both medical and financial research)
        - "Create a report on diabetes treatments" → research team (needs medical research first)
        - "Generate PDF from this data" → reporting team (document creation)
        
        Use your tools to make the routing decision."""
        
        self.agent = create_react_agent(
            llm, 
            [route_to_research_team, route_to_reporting_team],
            prompt=self.system_message
        )
    
    def __call__(self, state: AgentState) -> AgentState:
        messages = state["messages"]
        original_query = state.get("original_query", "")
        
        # Use LLM to make intelligent routing decision
        routing_messages = [
            SystemMessage(content=self.system_message),
            HumanMessage(content=f"Analyze this query and determine the best routing: {original_query}")
        ]
        
        result = self.agent.invoke({"messages": routing_messages})
        
        # Store routing decision for the conditional edge to use
        routing_response = result["messages"][-1].content
        
        return {
            **state,
            "routing_decision": routing_response,
            "messages": state["messages"]  # Keep original messages intact
        }


In [32]:
class Team1Agent:
    def __init__(self):
        self.name = "Team1_Research_Coordinator"
        self.system_message = """You are a Research Coordinator specialist.
        
        Analyze the research query and determine which specialist should handle it:
        
        1. For medical, pharmaceutical, clinical, health-related queries → route_to_medical_research
        2. For financial, economic, market, investment-related queries → route_to_financial_research
        
        Consider complex queries that might have multiple aspects. For example:
        - "Market impact of new Alzheimer's drug" → medical research (drug is primary focus)
        - "Financial performance of pharmaceutical companies" → financial research (financial analysis is primary)
        - "Clinical trial costs and budgets" → could be either, choose based on primary intent
        
        Use your judgment to determine the PRIMARY research need."""
        
        self.agent = create_react_agent(
            llm,
            [route_to_medical_research, route_to_financial_research],
            prompt=self.system_message
        )
    
    def __call__(self, state: AgentState) -> AgentState:
        original_query = state.get("original_query", "")
        
        # Use LLM to make intelligent routing decision
        routing_messages = [
            SystemMessage(content=self.system_message),
            HumanMessage(content=f"Analyze this research query and determine the best specialist: {original_query}")
        ]
        
        result = self.agent.invoke({"messages": routing_messages})
        
        # Store routing decision
        routing_response = result["messages"][-1].content
        
        return {
            **state,
            "routing_decision": routing_response
        }

In [33]:
class Team2Agent:
    def __init__(self):
        self.name = "Team2_Report_Coordinator"
    
    def __call__(self, state: AgentState) -> AgentState:
        # Simply pass through - routing logic is handled by conditional edges
        return {
            **state,
            "routing_decision": "team5"  # Always go to summary first
        }


In [34]:
class Team3Agent:
    def __init__(self):
        self.name = "Team3_Medical_Research"
        self.system_message = """You are a Medical Research Specialist.
        
        Use PubMed to search for peer-reviewed medical literature relevant to the user's query.
        
        Focus on:
        - Clinical studies and trials
        - Drug interactions and effects
        - Treatment protocols and outcomes
        - Medical research findings and evidence
        
        Provide comprehensive, evidence-based information with proper context.
        Search for specific topics mentioned in the user's query."""
        
        self.agent = create_react_agent(
            llm, 
            [tools["pubmed"]], 
            prompt=self.system_message
        )
    
    def __call__(self, state: AgentState) -> AgentState:
        # Use the original user query, not a generic one
        original_query = state.get("original_query", "")
        original_messages = state["messages"]
        
        # Pass the original query directly to the medical research agent
        research_messages = original_messages + [
            SystemMessage(content=self.system_message)
        ]
        
        result = self.agent.invoke({"messages": research_messages})
        
        # Store research results without routing decision
        research_results = state.get("research_results", {})
        research_results["medical_research"] = result["messages"][-1].content
        
        return {
            **state,
            "research_results": research_results,
            "task_completed": True
        }


In [35]:
class Team4Agent:
    def __init__(self):
        self.name = "Team4_Financial_Research"
        self.system_message = """You are a Financial Research Specialist.
        
        Use Tavily for current financial information and Wikipedia for financial concepts.
        
        Focus on:
        - Current market trends and analysis
        - Economic indicators and factors
        - Financial news and developments
        - Investment insights and analysis
        
        Provide current, comprehensive financial information relevant to the user's specific query.
        Use both tools as needed to give complete coverage."""
        
        self.agent = create_react_agent(
            llm, 
            [tools["tavily"], tools["wikipedia"]], 
            prompt=self.system_message
        )
    
    def __call__(self, state: AgentState) -> AgentState:
        # Use the original user query
        original_query = state.get("original_query", "")
        original_messages = state["messages"]
        
        # Pass the original query directly to the financial research agent
        research_messages = original_messages + [
            SystemMessage(content=self.system_message)
        ]
        
        result = self.agent.invoke({"messages": research_messages})
        
        # Store research results without routing decision
        research_results = state.get("research_results", {})
        research_results["financial_research"] = result["messages"][-1].content
        
        return {
            **state,
            "research_results": research_results,
            "task_completed": True
        }


In [36]:
class Team5Agent:
    def __init__(self):
        self.name = "Team5_Summary_Creator"
        self.system_message = """You are a Summary Creation Specialist.
        
        Create executive summaries from research results that are:
        - Clear and concise
        - Focused on key findings
        - Include actionable recommendations
        - Suitable for executive-level review
        
        Structure your summary to highlight the most important insights."""
        
        self.agent = create_react_agent(
            llm, 
            [create_summary_tool], 
            prompt=self.system_message
        )
    
    def __call__(self, state: AgentState) -> AgentState:
        research_results = state.get("research_results", {})
        original_query = state.get("original_query", "")
        
        # Prepare comprehensive research data
        if research_results:
            research_data = f"Original Query: {original_query}\n\n"
            research_data += "\n\n".join([
                f"=== {key.replace('_', ' ').upper()} ===\n{value}" 
                for key, value in research_results.items()
            ])
        else:
            research_data = f"Original Query: {original_query}\n\nNo research results available for processing."
        
        # Create summary using the tool
        summary_messages = [
            SystemMessage(content=self.system_message),
            HumanMessage(content=f"Create an executive summary from this research data:\n\n{research_data}")
        ]
        
        result = self.agent.invoke({"messages": summary_messages})
        
        # Store summary results
        summary_results = state.get("summary_results", {})
        summary_results["executive_summary"] = result["messages"][-1].content
        
        return {
            **state,
            "summary_results": summary_results,
            "task_completed": True
        }


In [37]:
# class Team6Agent:
#     def __init__(self):
#         self.name = "Team6_Document_Creator"
#         self.system_message = """You are a Document Creation Specialist.
        
#         Create professional documents (PDF and Word) from research content.
        
#         Focus on:
#         - Professional formatting and structure
#         - Clear document organization
#         - Appropriate titles and headers
#         - Comprehensive content inclusion
        
#         Create both PDF and Word versions of the document."""
        
#         self.agent = create_react_agent(
#             llm, 
#             [create_pdf_tool, create_word_tool], 
#             prompt=self.system_message
#         )
    
#     def __call__(self, state: AgentState) -> AgentState:
#         summary_results = state.get("summary_results", {})
#         research_results = state.get("research_results", {})
#         original_query = state.get("original_query", "")
        
#         # Create comprehensive document content
#         document_title = f"Research Report: {original_query}"
#         document_content = ""
        
#         if summary_results:
#             document_content += summary_results.get("executive_summary", "")
        
#         if research_results:
#             document_content += "\n\n=== DETAILED RESEARCH FINDINGS ===\n\n"
#             document_content += "\n\n".join([
#                 f"{key.replace('_', ' ').title()}:\n{value}" 
#                 for key, value in research_results.items()
#             ])
        
#         if not document_content:
#             document_content = f"Research report for query: {original_query}\n\nNo content available for document creation."
        
#         # Create documents
#         creation_messages = [
#             SystemMessage(content=self.system_message),
#             HumanMessage(content=f"Create both PDF and Word documents with title '{document_title}' and this content:\n\n{document_content}")
#         ]
        
#         result = self.agent.invoke({"messages": creation_messages})
        
#         return {
#             **state,
#             "final_output": result["messages"][-1].content,
#             "task_completed": True
#         }
class Team6Agent:
    def __init__(self):
        # We no longer need a ReAct agent here because we will call the tools directly.
        self.name = "Team6_Document_Creator"

    def __call__(self, state: AgentState) -> AgentState:
        print("--- 📄 Document Creator at work (Direct Calling) ---")
        summary_results = state.get("summary_results", {})
        research_results = state.get("research_results", {})
        original_query = state.get("original_query", "")

        # 1. Prepare the content (this part of your code is perfect)
        document_title = f"Research Report: {original_query}"
        document_content = ""
        if summary_results:
            document_content += summary_results.get("executive_summary", "")
        if research_results:
            document_content += "\n\n=== DETAILED RESEARCH FINDINGS ===\n\n"
            document_content += "\n\n".join([
                f"{key.replace('_', ' ').title()}:\n{value}"
                for key, value in research_results.items()
            ])
        if not document_content:
            document_content = "No content available for document creation."

        # 2. Call the tools directly using Python - NO LLM NEEDED
        pdf_result = create_pdf_tool.invoke({
            "title": document_title,
            "content": document_content
        })
        print(f"PDF Tool Output: {pdf_result}")

        word_result = create_word_tool.invoke({
            "title": document_title,
            "content": document_content
        })
        print(f"Word Tool Output: {word_result}")

        # 3. Combine the results into the final output
        final_output_str = f"Document generation complete.\n- {pdf_result}\n- {word_result}"

        return {
            **state,
            "final_output": final_output_str,
            "task_completed": True
        }


In [38]:
# Intelligent routing functions for conditional edges
def route_from_supervisor(state: AgentState) -> Literal["team1", "team2"]:
    """Route based on supervisor's intelligent decision"""
    routing_decision = state.get("routing_decision", "")
    
    if "team1" in routing_decision:
        return "team1"
    elif "team2" in routing_decision:
        return "team2"
    else:
        return "team1"  # Default to research

In [39]:
def route_from_team1(state: AgentState) -> Literal["team3", "team4"]:
    """Route based on Team1's intelligent decision"""
    routing_decision = state.get("routing_decision", "")
    
    if "team3" in routing_decision:
        return "team3"
    elif "team4" in routing_decision:
        return "team4"
    else:
        return "team3"  # Default to medical

In [40]:
def route_from_team2(state: AgentState) -> Literal["team5"]:
    """Always route to summary creation first"""
    return "team5"

def route_from_team5(state: AgentState) -> Literal["team6"]:
    """Always route to document creation after summary"""
    return "team6"

def route_to_end(state: AgentState) -> Literal["END"]:
    """Route to end when task is completed"""
    return END

def route_research_to_reporting(state: AgentState) -> Literal["team2"]:
    """Route from research teams to reporting team"""
    return "team2"


In [41]:
# Create the enhanced multi-agent graph
def create_enhanced_multi_agent_graph():
    """Create the improved multi-agent system with intelligent routing"""
    
    # Initialize agents
    supervisor = SupervisorAgent()
    team1 = Team1Agent()
    team2 = Team2Agent()
    team3 = Team3Agent()
    team4 = Team4Agent()
    team5 = Team5Agent()
    team6 = Team6Agent()
    
    # Create the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes - each node only handles its specific task
    workflow.add_node("supervisor", supervisor)
    workflow.add_node("team1", team1)
    workflow.add_node("team2", team2)
    workflow.add_node("team3", team3)
    workflow.add_node("team4", team4)
    workflow.add_node("team5", team5)
    workflow.add_node("team6", team6)
    
    # Add edges - routing logic is centralized here
    workflow.add_edge(START, "supervisor")
    workflow.add_conditional_edges("supervisor", route_from_supervisor)
    workflow.add_conditional_edges("team1", route_from_team1)
    workflow.add_conditional_edges("team2", route_from_team2)
    workflow.add_conditional_edges("team3", route_research_to_reporting)
    workflow.add_conditional_edges("team4", route_research_to_reporting)
    workflow.add_conditional_edges("team5", route_from_team5)
    workflow.add_edge("team6", END)
    
    # Compile with memory
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

In [42]:
"""Run examples with the enhanced system"""
app = create_enhanced_multi_agent_graph()

# Example 1: Complex query that could be routed multiple ways
print("=== Enhanced Example 1: Complex Query ===")
config = {"configurable": {"thread_id": "complex_query_1"}}

complex_query = "Analyze the market impact of new Alzheimer's drug treatments on pharmaceutical companies"

state = {
    "messages": [HumanMessage(content=complex_query)],
    "original_query": complex_query,
    "research_results": {},
    "summary_results": {},
    "final_output": "",
    "routing_decision": None,
    "task_completed": False
}

try:
    result = app.invoke(state, config)
    print(f"Query: {complex_query}")
    print(f"Research Results Keys: {list(result['research_results'].keys())}")
    print(f"Final Output: {result['final_output'][:200]}...")
except Exception as e:
    print(f"Error: {e}")


=== Enhanced Example 1: Complex Query ===
--- 📄 Document Creator at work (Direct Calling) ---
PDF Tool Output: PDF successfully created: /var/folders/bl/xc5d5vc16wx60khkhwlcs3m40000gn/T/tmpkyjdh784.pdf
Title: Research Report: Analyze the market impact of new Alzheimer's drug treatments on pharmaceutical companies
Content length: 618 characters
Word Tool Output: Word document successfully created: /var/folders/bl/xc5d5vc16wx60khkhwlcs3m40000gn/T/tmpvd0sw1o0.docx
Title: Research Report: Analyze the market impact of new Alzheimer's drug treatments on pharmaceutical companies
Content length: 618 characters
Query: Analyze the market impact of new Alzheimer's drug treatments on pharmaceutical companies
Research Results Keys: ['medical_research']
Final Output: Document generation complete.
- PDF successfully created: /var/folders/bl/xc5d5vc16wx60khkhwlcs3m40000gn/T/tmpkyjdh784.pdf
Title: Research Report: Analyze the market impact of new Alzheimer's drug tre...
